size of dicom image

In [2]:
import os
import pydicom
import numpy as np

# Set the path to the folder containing the subfolders
# folder_path = '/root/autodl-tmp/image_arterial'
folder_path = '/root/autodl-tmp/image_delay'

# Get a sorted list of subfolders
subfolders = sorted(os.listdir(folder_path))

# Iterate over the sorted subfolders
for subfolder_name in subfolders:
    subfolder_path = os.path.join(folder_path, subfolder_name)
    
    # Create a list to store the DICOM images
    dicom_images = []
    
    # Iterate over the files in the subfolder
    for file_name in os.listdir(subfolder_path):
        file_path = os.path.join(subfolder_path, file_name)
        
        # Check if the file is a DICOM file
        if file_path.endswith('.dcm'):
            # Read the DICOM file
            dicom_image = pydicom.dcmread(file_path)
            
            # Append the DICOM image to the list
            dicom_images.append(dicom_image)
    
    # Check if DICOM images were found in the subfolder
    if len(dicom_images) > 0:
        # Stack the DICOM images into a 3D array
        stacked_image = np.stack([dicom.pixel_array for dicom in dicom_images])
        
        # Print the shape of the stacked image
        print(f"Subfolder: {subfolder_name[:7]}, Shape: {stacked_image.shape}")

Subfolder: MRI0109, Shape: (120, 512, 512)
Subfolder: MRI0150, Shape: (80, 512, 512)
Subfolder: Mri0041, Shape: (96, 512, 512)
Subfolder: Mri0059, Shape: (149, 640, 640)
Subfolder: Mri0061, Shape: (96, 512, 512)
Subfolder: Mri0062, Shape: (136, 640, 640)
Subfolder: Mri0064, Shape: (180, 768, 768)
Subfolder: Mri0113, Shape: (187, 800, 800)
Subfolder: Mri0114, Shape: (112, 512, 512)
Subfolder: Mri0122, Shape: (59, 512, 512)
Subfolder: Mri0179, Shape: (149, 640, 640)
Subfolder: Mri0209, Shape: (167, 640, 640)
Subfolder: Mri0322, Shape: (112, 512, 512)
Subfolder: Mri0330, Shape: (97, 512, 512)
Subfolder: Mri0340, Shape: (149, 640, 640)
Subfolder: Mri0461, Shape: (153, 640, 640)
Subfolder: Mri0468, Shape: (112, 512, 512)
Subfolder: Mri0529, Shape: (149, 768, 768)
Subfolder: Mri0547, Shape: (131, 640, 640)
Subfolder: Mri0557, Shape: (149, 640, 640)
Subfolder: Mri0569, Shape: (96, 512, 512)
Subfolder: Mri0686, Shape: (120, 512, 512)
Subfolder: Mri0695, Shape: (149, 768, 768)
Subfolder: Mri070

resize dicom images in folders

In [ ]:
from glob import glob
import os
import pydicom
import numpy as np
import cv2

# root_folder = '/root/autodl-tmp/image_delay'  
# output_folder = '/root/autodl-tmp/image_delay_512'  
root_folder = "/root/autodl-tmp/image_arterial"
output_folder = "/root/autodl-tmp/image_arterial_512"

new_size = (512, 512)  # Specify the new size for resizing

for root, dirs, files in os.walk(root_folder):
    for dcm_name in files:
        if dcm_name.endswith('.dcm'):  # Process only DICOM files
            dcm_path = os.path.join(root, dcm_name)
            ds = pydicom.dcmread(dcm_path)
            array = ds.pixel_array

            resize_array = cv2.resize(array, new_size, interpolation=cv2.INTER_LINEAR)

            ds.Rows = new_size[0]
            ds.Columns = new_size[1]

            ds.PixelData = resize_array

            subfolder_path = os.path.join(output_folder, os.path.relpath(root, root_folder))
            os.makedirs(subfolder_path, exist_ok=True)  # Create subfolder if it doesn't exist

            output_path = os.path.join(subfolder_path, dcm_name)
            ds.save_as(output_path)

resize nii.gz ground truth

In [ ]:
import os
import skimage.transform as skTrans
import nibabel as nib
import numpy as np

input_folder = "/root/autodl-tmp/P3_delay"  # Path to the folder containing the NIfTI files
output_folder = "/root/autodl-tmp/P3_delay_512"  # Path to the output folder

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Get a list of file names in the input folder
file_names = os.listdir(input_folder)

# Process each file
for file_name in file_names:
    if file_name.endswith(".nii.gz"):
        # Load the image from the input file
        file_path = os.path.join(input_folder, file_name)
        im = nib.load(file_path).get_fdata()

        # Perform the resizing operation
        result = skTrans.resize(im, (512, 512, im.shape[2]), order=1, preserve_range=True)

        # Apply flip operation to correct orientation
        result = np.flip(result, axis=0)  # Flip along the first axis (up-down flip)
        result = np.flip(result, axis=1)  # Flip along the second axis (left-right flip)

        # Save the result to the output folder
        output_path = os.path.join(output_folder, file_name)
        nib.save(nib.Nifti1Image(result, affine=None), output_path)

convert dcm to nii

In [ ]:
dcm2niix -o /root/autodl-tmp/image_arterial_512converted -z y /root/autodl-tmp/image_arterial_512

In [ ]:
!pip install dcm2niix

In [1]:
import os

def dcm2nii(dicom_path, nii_path):
    # Create the target folder if it doesn't exist
    if not os.path.exists(nii_path):
        os.makedirs(nii_path)
    
    for folder in os.listdir(dicom_path):
        src_path = os.path.join(dicom_path, folder)
        dst_path = os.path.join(nii_path, f"{folder}.nii.gz")
        os.system(f"dcm2niix -o {nii_path} {src_path}")
        for file in os.listdir(nii_path):
            if file.endswith(".nii.gz"):
                new_file = os.path.join(nii_path, f"{folder}.nii.gz")
                os.rename(os.path.join(nii_path, file), new_file)

dicom_path = "/root/autodl-tmp/nnUNet/nnUNet_raw/Dataset151_Liverdelay/imagesTs"
nii_path = "/root/autodl-tmp/nnUNet/nnUNet_raw/Dataset151_Liverdelay/imagesTs"
# dicom_path = "/root/autodl-tmp/image_delay"
# nii_path = "autodl-tmp/image_delay_nii"

dcm2nii(dicom_path, nii_path)

Chris Rorden's dcm2niiX version v1.0.20220505  GCC9.4.0 x86-64 (64-bit Linux)
Found 149 DICOM file(s)
Philips Scaling Values RS:RI:SS = 1.88767:0:0.00203674 (see PMC3998685)
Convert 149 DICOM as /root/autodl-tmp/nnUNet/nnUNet_raw/Dataset151_Liverdelay/imagesTs/Mri0039_Arterial_BH_1301_WIP_Arterial_BH_SENSE_20170328091604_1301 (672x672x149x1)
Conversion required 0.704543 seconds (0.704495 for core code).
Chris Rorden's dcm2niiX version v1.0.20220505  GCC9.4.0 x86-64 (64-bit Linux)


Error: Unable to find any DICOM images in /root/autodl-tmp/nnUNet/nnUNet_raw/Dataset151_Liverdelay/imagesTs/.ipynb_checkpoints (or subfolders 5 deep)


Chris Rorden's dcm2niiX version v1.0.20220505  GCC9.4.0 x86-64 (64-bit Linux)
Found 104 DICOM file(s)
Resolved discrepancy between tags (0020,1002; 0021,104F; 0054,0081)
Convert 104 DICOM as /root/autodl-tmp/nnUNet/nnUNet_raw/Dataset151_Liverdelay/imagesTs/Mri0126_LIVER_LAVA_AXIAL_ARTERIAL_PHASE_12_HKU_ABDOMEN_P+C_20190528124831_12 (512x512x104x1)
Conversion required 0.314071 seconds (0.314033 for core code).
Chris Rorden's dcm2niiX version v1.0.20220505  GCC9.4.0 x86-64 (64-bit Linux)
Found 120 DICOM file(s)
Resolved discrepancy between tags (0020,1002; 0021,104F; 0054,0081)
Convert 120 DICOM as /root/autodl-tmp/nnUNet/nnUNet_raw/Dataset151_Liverdelay/imagesTs/Mri0452_LIVER_LAVA_AXIAL_ARTERIAL_PHASE_11_HKU_ABDOMEN_P+C_20200311104223_11 (512x512x120x1)
Conversion required 0.341461 seconds (0.341381 for core code).


Flip nii.gz mask

In [1]:
import os
import nibabel as nib
import numpy as np

# input_folder_path = "/root/autodl-tmp/P2_arterial"
# output_folder_path = "/root/autodl-tmp/P2_arterial_f"

# input_folder_path = "/root/autodl-tmp/P3_delay"
# output_folder_path = "/root/autodl-tmp/P3_delay_f"

input_folder_path = "/root/autodl-tmp/fyp2_mask_result/boxcombined"
output_folder_path = "/root/autodl-tmp/fyp2_mask_result/f_boxcombined"

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

# Get a list of all NIfTI files in the input folder
nifti_files = [f for f in os.listdir(input_folder_path) if f.endswith('.nii.gz')]

# Loop through each NIfTI file
for nifti_file in nifti_files:
    # Load the NIfTI image
    input_file_path = os.path.join(input_folder_path, nifti_file)
    image = nib.load(input_file_path)

    # Get the image data as a NumPy array
    image_data = image.get_fdata()

    # Flip the image along the first axis (up-down flip)
    flipped_image = np.flip(image_data, axis=0)

    # Flip the flipped image along the second axis (left-right flip)
    final_image = np.flip(flipped_image, axis=1)

    # Create a new NIfTI image with the flipped data
    flipped_nifti = nib.Nifti1Image(final_image, image.affine)

    # Construct the output file path for the flipped image
    output_file_path = os.path.join(output_folder_path, "flipped_" + nifti_file)

    # Save the flipped NIfTI image to the output folder
    nib.save(flipped_nifti, output_file_path)

nii to nii.gz

In [5]:
import os
import gzip
import shutil

def compress_folder_nii_to_nii_gz(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for root, dirs, files in os.walk(input_folder):
        for file in files:
            if file.endswith('.nii'):
                input_file = os.path.join(root, file)
                output_file = os.path.join(output_folder, file + '.gz')
                with open(input_file, 'rb') as f_in:
                    with gzip.open(output_file, 'wb') as f_out:
                        shutil.copyfileobj(f_in, f_out)

input_folder = '/root/autodl-tmp/image_arterial_nii'
output_folder = '/root/autodl-tmp/image_arterial_niigz'
# input_folder = '/root/autodl-tmp/image_delay_nii'
# output_folder = '/root/autodl-tmp/image_delay_niigz'

compress_folder_nii_to_nii_gz(input_folder, output_folder)

verify size of nii

In [7]:
import os
import nibabel as nib

# folder_path = '/root/autodl-tmp/P2_arterial_f'
# folder_path = '/root/autodl-tmp/image_arterial_niigz'
# folder_path = '/root/autodl-tmp/P3_delay'
# folder_path = '/root/autodl-tmp/image_delay_niigz'
# folder_path = "/root/autodl-tmp/nnUNet/nnUNet_raw/Dataset150_Liverarterial/imagesTr"
# folder_path = "/root/autodl-tmp/nnUNet/nnUNet_raw/Dataset150_Liverarterial/labelsTr"
folder_path = "/root/autodl-tmp/fyp2_mask_result/medsam"

file_list = os.listdir(folder_path)
file_list.sort()  # Sort the file names

for file_name in file_list:
    file_path = os.path.join(folder_path, file_name)

    if file_name.endswith('.nii') or file_name.endswith('.nii.gz'):
        image = nib.load(file_path)
        image_shape = image.shape

        # Print the first 7 characters of the file name
        print(f"File: {file_name[:17]}, Shape: {image_shape}")

File: Liverarterial_001, Shape: (512, 512, 96)
File: Liverarterial_002, Shape: (576, 576, 149)
File: Liverarterial_003, Shape: (512, 512, 96)
File: Liverarterial_004, Shape: (576, 576, 136)
File: Liverarterial_005, Shape: (672, 672, 180)
File: Liverarterial_006, Shape: (512, 512, 120)
File: Liverarterial_007, Shape: (672, 672, 187)
File: Liverarterial_008, Shape: (512, 512, 112)
File: Liverarterial_009, Shape: (576, 576, 149)
File: Liverarterial_010, Shape: (512, 512, 112)
File: Liverarterial_011, Shape: (512, 512, 96)
File: Liverarterial_012, Shape: (576, 576, 149)
File: Liverarterial_013, Shape: (576, 576, 153)
File: Liverarterial_014, Shape: (512, 512, 112)
File: Liverarterial_015, Shape: (672, 672, 149)
File: Liverarterial_016, Shape: (576, 576, 131)
File: Liverarterial_017, Shape: (576, 576, 149)
File: Liverarterial_018, Shape: (512, 512, 96)
File: Liverarterial_019, Shape: (512, 512, 120)
File: Liverarterial_020, Shape: (512, 512, 120)
File: Liverarterial_021, Shape: (576, 576, 1

count how many case of dcm

In [ ]:
import os

def count_subfolders(folder_path):
    subfolder_count = 0

    for root, dirs, files in os.walk(folder_path):
        subfolder_count += len(dirs)

    return subfolder_count

# Example usage
folder_path = "/root/autodl-tmp/image_delay"
# folder_path = "/root/autodl-tmp/image_arterial"
subfolder_count = count_subfolders(folder_path)
print("Number of subfolders:", subfolder_count)

count how many nii files

In [5]:
import os

def count_nii_gz_files(directory):
    count = 0
    for filename in os.listdir(directory):
        if filename.endswith(".nii.gz"):
            count += 1
    return count

# directory_path = "/root/autodl-tmp/P3_delay"
# directory_path = '/root/autodl-tmp/image_delay_niigz'
directory_path = '/root/autodl-tmp/fyp2_mask_result/pointcombined'
# directory_path = "/root/autodl-tmp/P2_arterial"

file_count = count_nii_gz_files(directory_path)
print("Number of nii.gz files:", file_count)

Number of nii.gz files: 27


Rename files

In [4]:
import os

# folder_path = "/root/autodl-tmp/nnUNet/nnUNet_raw/Dataset150_Liverarterial/imagesTr"
# folder_path = "/root/autodl-tmp/nnUNet/nnUNet_raw/Dataset150_Liverarterial/labelsTr"
folder_path = "/root/autodl-tmp/fyp2_mask_result/pointcombined"

prefix = "Liverarterial_"

# Get a list of files in the folder
file_list = os.listdir(folder_path)

# Filter the files by extension and sort them
nii_files = sorted([filename for filename in file_list if filename.endswith(".nii.gz")])

# Iterate over the files and rename them
for i, filename in enumerate(nii_files, start=1):
    # new_filename = f"{prefix}{str(i).zfill(3)}_0000.nii.gz"
    new_filename = f"{prefix}{str(i).zfill(3)}.nii.gz"
    old_filepath = os.path.join(folder_path, filename)
    new_filepath = os.path.join(folder_path, new_filename)
    os.rename(old_filepath, new_filepath)

change label values to match orders

In [1]:
import os
import nibabel as nib
import numpy as np

# folder_path = 'autodl-tmp/P2_arterial_f'
# folder_path = '/root/autodl-tmp/P2_arterial_f_label'
# folder_path = '/root/autodl-tmp/nnUNet/nnUNet_raw/Dataset150_Liverarterial/labelsTr'

folder_path = '/root/autodl-tmp/P3_delay'

file_list = sorted([filename for filename in os.listdir(folder_path) if filename.endswith('.nii.gz')])

for filename in file_list:
    file_path = os.path.join(folder_path, filename)
    img = nib.load(file_path)
    data = img.get_fdata()

    unique_values = np.unique(data)
    print("Values in", filename[:27])
    for value in unique_values:
        print(value)
    print()

Values in Mri0041_DELAY_3_MINS_14.nii
0.0
2.0
4.0

Values in Mri0059_Delayed_3_minsThriv
0.0
4.0

Values in Mri0061_LAVA_AXIAL_DELAY_3_
0.0
4.0

Values in Mri0062_Delayed_3_minsThriv
0.0
4.0

Values in Mri0064_Delayed_3_minsThriv
0.0
2.0

Values in Mri0109_LIVER_LAVA_AXIAL_DE
0.0
2.0

Values in Mri0113_Delayed_3_minsThriv
0.0
4.0

Values in Mri0114_LIVER_LAVA_AXIAL_PO
0.0
4.0

Values in Mri0122_DELAYED_3_MINS_TRAN
0.0
4.0

Values in Mri0150_LAVA_AXIAL_PORTOVEN
0.0
2.0
4.0

Values in Mri0179_Delayed_3_minsThriv
0.0
2.0
4.0

Values in Mri0209_Delayed_3_minsThriv
0.0
5.0

Values in Mri0322_LIVER_LAVA_AXIAL_DE
0.0
2.0

Values in Mri0330_LIVER_LAVA_AXIAL_DE
0.0
2.0

Values in Mri0340_Delayed_3_minsThriv
0.0
2.0

Values in Mri0461_Delayed_3_minsThriv
0.0
4.0

Values in Mri0468_LIVER_LAVA_AXIAL_DE
0.0
4.0

Values in Mri0529_Delayed_3_minsThriv
0.0
4.0

Values in Mri0547_Delayed_3_minsThriv
0.0
4.0

Values in Mri0557_Delayed_3_minsThriv
0.0
2.0

Values in Mri0569_LIVER_LAVA_AXIAL_DE
0.0
2.0

V

In [4]:
import os
import nibabel as nib
import numpy as np

# input_folder_path = '/root/autodl-tmp/P2_arterial_f' 
# output_folder_path = '/root/autodl-tmp/P2_arterial_f_label'  
input_folder_path = '/root/autodl-tmp/P3_delay_f'
output_folder_path = '/root/autodl-tmp/P3_delay_f_label'

# Create output folder if it doesn't exist
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

for filename in os.listdir(input_folder_path):
    if filename.endswith('.nii.gz'):
        input_file_path = os.path.join(input_folder_path, filename)
        output_file_path = os.path.join(output_folder_path, 'modified_' + filename)

        img = nib.load(input_file_path)
        data = img.get_fdata()
        
        # Change specific values
        data[data == 4] = 3
        
        # Save the modified data to the output file
        modified_img = nib.Nifti1Image(data, img.affine, img.header)
        nib.save(modified_img, output_file_path)
        
        print("Modified file saved as", output_file_path)

Modified file saved as /root/autodl-tmp/P3_delay_f_label/modified_flipped_Mri0041_DELAY_3_MINS_14.nii.gz
Modified file saved as /root/autodl-tmp/P3_delay_f_label/modified_flipped_Mri0719_Delayed_3_minsThrive_LIVER_1301.nii.gz
Modified file saved as /root/autodl-tmp/P3_delay_f_label/modified_flipped_Mri0059_Delayed_3_minsThrive_LIVER_1601.nii.gz
Modified file saved as /root/autodl-tmp/P3_delay_f_label/modified_flipped_Mri0061_LAVA_AXIAL_DELAY_3_MINS_16.nii.gz
Modified file saved as /root/autodl-tmp/P3_delay_f_label/modified_flipped_Mri0754_Delayed_3_minsThrive_LIVER_1301.nii.gz
Modified file saved as /root/autodl-tmp/P3_delay_f_label/modified_flipped_Mri0062_Delayed_3_minsThrive_LIVER_1601.nii.gz
Modified file saved as /root/autodl-tmp/P3_delay_f_label/modified_flipped_Mri0899_LIVER_LAVA_AXIAL_DELAY_3MIN_12.nii.gz
Modified file saved as /root/autodl-tmp/P3_delay_f_label/modified_flipped_Mri0064_Delayed_3_minsThrive_LIVER_1601.nii.gz
Modified file saved as /root/autodl-tmp/P3_delay_f_lab

Manual split of training and validation set

In [ ]:
import os

os.system("/usr/bin/shutdown")

In [17]:
from batchgenerators.utilities.file_and_folder_operations import load_json
splits = load_json('/root/autodl-tmp/nnUNet/nnUNet_preprocessed/Dataset150_Liverarterial/splits_final.json')

print(len(splits))
print(splits[0].keys())
print(len(splits[0]['train']))
print(splits[0])

5
dict_keys(['train', 'val'])
25
{'train': ['Liverarterial_001', 'Liverarterial_002', 'Liverarterial_003', 'Liverarterial_005', 'Liverarterial_007', 'Liverarterial_008', 'Liverarterial_009', 'Liverarterial_010', 'Liverarterial_011', 'Liverarterial_012', 'Liverarterial_013', 'Liverarterial_014', 'Liverarterial_015', 'Liverarterial_016', 'Liverarterial_017', 'Liverarterial_018', 'Liverarterial_019', 'Liverarterial_020', 'Liverarterial_021', 'Liverarterial_022', 'Liverarterial_024', 'Liverarterial_025', 'Liverarterial_026', 'Liverarterial_027', 'Liverarterial_028'], 'val': ['Liverarterial_004', 'Liverarterial_006', 'Liverarterial_023']}
